In [1]:
import numpy as np
import pandas as pd
import utils

import torch
import torch.nn as nn
from torch.utils import data

In [3]:
df = pd.read_csv("rsync_folder/total_df.csv")
df.tail()

,path,text,score,vader,HFSID,UPDATED_DAY
651357,/mnt/processed/edgar_footnote_partition/10-K/2...,in addition to the 2010 property acquisitions ...,0.500000,0.0000,406200101,20120224
651358,/mnt/processed/edgar_footnote_partition/10-K/2...,january 2010 the company complete the sale o...,0.948070,0.0772,406200101,20120224
651359,/mnt/processed/edgar_footnote_partition/10-K/2...,a substantial portion of the companys lease re...,0.780377,0.3612,406200101,20120224
651360,/mnt/processed/edgar_footnote_partition/10-K/2...,portfolio by region: the table below reflect t...,0.500000,0.0000,406200101,20120224
651361,/mnt/processed/edgar_footnote_partition/10-K/2...,net income allocable to common shareholder wa ...,0.878319,0.8176,406200101,20120224


In [4]:
use_df = df[(df.score <= 0.25) | (df.score >= 0.75) ]
use_df[use_df.score > 0.5].shape[0],use_df[use_df.score < 0.5].shape[0]

use_df['label'] = [1 if i > 0.5 else 0 for i in use_df.score]
max_size = use_df['label'].value_counts().max()

(280541, 133922)

/usr/local/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
%time corpus,count = np.unique([word for sent in use_df.text for word in sent.split(" ")],return_counts=True)
len(corpus)

In [5]:
lemma_dict = {'pad':0,'unk':1}
for word in corpus : 
    lemma_dict[word] = len(lemma_dict)
    
word_to_freq_dict = dict(zip(corpus,count))

use_df['indexed_text'] = use_df.text.str.split(" ").apply(lambda x : list(map(lemma_dict.get,x)))

In [ ]:
def padding(arr) : 
    arr = arr + [0] * (100-len(arr)) 
    return arr

use_df['indexed_text'] = use_df.indexed_text.apply(padding)
use_df.indexed_text.apply(len).unique()

In [9]:
lst = [use_df]
for class_index, group in use_df.groupby('label'):
    lst.append(group.sample(max_size-len(group), replace=True))
new_use_df = pd.concat(lst)

In [13]:
new_use_df.to_csv("rsync_folder/new_use_df.csv",index=False)

In [13]:
pd.to_pickle(lemma_dict,"lemma_dict.pkl")
pd.to_pickle(word_to_freq_dict,"word_to_freq_dict.pkl")

In [1]:
import pickle

In [4]:
import pandas as pd

In [6]:
lemma_dict = pd.read_pickle('rsync_folder/lemma_dict.pkl')

In [8]:
with open('rsync_folder/lemma_dict.pkl','wb') as f :
    pickle.dump(lemma_dict,f,protocol=2)

_______

In [14]:
train_X = torch.tensor(list(new_use_df.indexed_text.apply(lambda x : np.array(x)).values))
train_X = train_X.type(torch.float)

train_y = torch.tensor(new_use_df.label.values)

In [15]:
torch.save(train_X,'train_X.pt')
torch.save(train_y,'train_y.pt')

______

In [15]:
test_df = pd.read_csv("data/test_result_df.csv")

In [26]:
tmp_df = utils.assign_ii_di(test_df)

In [ ]:
use_df = tmp_df[['parsed_text','HFSID','UPDATED_DAY']]
use_df.rename(columns={'parsed_text':'text'},inplace=True)

In [60]:
result_df,train_X,_ = utils.indexing(use_df,lemma_dict,train=False)

In [63]:
result_df.to_csv('rsync_folder/test_df.csv',index=False)
torch.save(train_X,'rsync_folder/test_X.pt')